In [302]:
! pip install predictionguard

In [303]:
import os
import random

from predictionguard import PredictionGuard
from getpass import getpass
import pandas as pd

In [304]:
pg_access_token = getpass('Enter your Prediction Guard access api key: ')
os.environ['PREDICTIONGUARD_API_KEY'] = pg_access_token

In [305]:
client = PredictionGuard()

# Load data

In [306]:
data = pd.read_csv('C:/Users/evhai/OneDrive/test.csv')

In [307]:
data.loc[0]


Id     a2c83973-8e81-45c9-bb84-37fa8a8b637c
q1                                        3
q2                                      NaN
q3                                        0
q4                                        1
q5                                        1
q6                                        1
q7                                        4
q8                                        4
q9                                        1
q10                                       0
q11                                       3
q12                                       2
q13                                       0
q14                                       3
q15                                       1
q16                                       1
q17                                       0
q18                                       0
q19                                       0
q20                                       0
q21                                     NaN
q22                             

In [308]:
questions = {
 'q1': 'Please share all of the reasons you chose to attend a TAPS seminar.',
 'q2': 'How did you find out about the TAPS seminar?',
 'q3': 'This seminar helped me to feel socially connected',
 'q4': 'This seminar taught me new ways to cope with my grief',
 'q5': 'This seminar helped me to better understand my grief',
 'q6': 'This seminar has given me hope for the future',
 'q7': 'Based on your experience at the TAPS seminar, how likely are you to continue to connect with TAPS virtually / at in-person programs?',
 'q8': 'Based on your experience at the TAPS seminar, how likely are you to continue to bring your child(ren) to connect with TAPS at in-person programs?',
 'q9': 'Rate your level of agreement: I have a positive outlook toward life',
 'q10': 'Rate your level of agreement: I have short and/or long range goals',
 'q11': 'Rate your level of agreement: I feel all alone',
 'q12': 'Rate your level of agreement: I can see possibilities in the midst of difficulties',
 'q13': 'Rate your level of agreement: I have faith that gives me comfort',
 'q14': 'Rate your level of agreement: I feel scared about my future',
 'q15': 'Rate your level of agreement: I can recall happy/joyful times',
 'q16': 'Rate your level of agreement: I have deep inner strength',
 'q17': 'Rate your level of agreement: I am able to give and receive care/love',
 'q18': 'Rate your level of agreement: I have a sense of direction',
 'q19': 'Rate your level of agreement: I believe that each day has potential',
 'q20': 'Rate your level of agreement: I feel my life has value and worth',
 'q21': 'Please share with TAPS your favorite moment of the weekend? Did you have a breakthrough moment this weekend you would like to share?',
 'q22': 'Please share any additional feedback you have regarding your TAPS Seminar experience.'
}

In [309]:
rating_reference = {
    0: "Extremely likely / Very likely",
    1: "Somewhat likely",
    2: "Neither likely nor unlikely / Neutral",
    3: "Somewhat unlikely",
    4: "Extremely unlikely / Not at all likely"
}

# Basic prompting/ prompt engineering: Label directly

In [310]:
prompt = """Below are a set of answers to questions from a survey of people going through a grief journey after losing a family member.
Classify the person that answered these questions into one of the following categories representing where they are in their grief journey:

1. "Immediate Grief, Shock & Emotion"
2. "Navigating Family Relationships"
3. "Learning to Process Grief"
4. "Moments That Matter"
5. "Feeling Immersed, Connected & Seen"
6. "New Growth & Purpose"

Respond with a number representing the appropriate category. Respond with only the number and no other text.

Answers to survey questions:
{qanda}

Category number:
"""

In [311]:
survey_num = 0
qanda = ""
for i in range(9, 21):
  qanda += questions[f'q{i}'].split('agreement: ')[-1] + ": " + rating_reference[data.loc[survey_num][f'q{i}']] + "\n"
qanda = qanda.strip()

In [312]:
print(prompt.format(qanda=qanda))

Below are a set of answers to questions from a survey of people going through a grief journey after losing a family member.
Classify the person that answered these questions into one of the following categories representing where they are in their grief journey:

1. "Immediate Grief, Shock & Emotion"
2. "Navigating Family Relationships"
3. "Learning to Process Grief"
4. "Moments That Matter"
5. "Feeling Immersed, Connected & Seen"
6. "New Growth & Purpose"

Respond with a number representing the appropriate category. Respond with only the number and no other text.

Answers to survey questions:
I have a positive outlook toward life: Somewhat likely
I have short and/or long range goals: Extremely likely / Very likely
I feel all alone: Somewhat unlikely
I can see possibilities in the midst of difficulties: Neither likely nor unlikely / Neutral
I have faith that gives me comfort: Extremely likely / Very likely
I feel scared about my future: Somewhat unlikely
I can recall happy/joyful times

In [313]:
messages = [
    {
        "role": "user",
        "content": prompt.format(qanda=qanda)
    }
]

result = client.chat.completions.create(
    model="Hermes-2-Pro-Llama-3-8B",
    messages=messages,
    max_tokens=1000
)
result['choices'][0]['message']['content']

'5'

# Data augmentation: Inject definitions

---



In [314]:
journey_map = {
    "Immediate Grief, Shock & Emotion": {
        "number": 1,
        "description": "Overwhelmed, loss of purpose; shock and trauma emotions (isolation) present and challenging to understand. Individuals may struggle to deal with family responsibilities alone. Surviving Child: Feeling disconnected without guidance and attention from grieving adults.",
        "support_elements": "Mission critical. 24/7 National Military Survivor Helpline. Ongoing Survivor Care Team."
    },
    "Navigating Family Relationships": {
        "number": 2,
        "description": "Experiencing tension between individuals within the family unit; lack of support from family members. Surviving Family Unit: Perception of other family members’ grief experience. Each family member may be at different phases of their grief journey.",
        "support_elements": "Guidance & Acknowledgement. Catered resources."
    },
    "Learning to Process Grief": {
        "number": 3,
        "description": "Experiencing grief and learning to process those emotions. Surviving Child: Seeks guidance and acknowledgment of grief; benefit from opportunities to open up and process with kids in similar situations to normalize emotions.",
        "support_elements": "Community support. Safe space to process emotions."
    },
    "Moments That Matter": {
        "number": 4,
        "description": "Renewed experience of grief around anniversaries of loss, holidays, and special moments. Surviving Family Unit: Navigating special moments (sports, school achievements, moments that matter).",
        "support_elements": "Continued grief support."
    },
    "Feeling Immersed, Connected & Seen": {
        "number": 5,
        "description": "Finding new purpose and goals to begin moving towards Positive Integration. Surviving Family Unit: Connected to a broader community; support system; not the only person/family experiencing loss.",
        "support_elements": "Maintain community. Safe space to process emotions."
    },
    "New Growth & Purpose": {
        "number": 6,
        "description": "Healthy point in grief journey; feeling capable to help others and a desire to do so. Surviving Family Unit: Ready to give back to the TAPS community through mentorship programs, volunteering at charity drives & events, etc.",
        "support_elements": "Maintain family connection/healing. Desire to support others."
    }
}

In [315]:
prompt = """Below are a set of answers to questions from a survey of people going through a grief journey after losing a family member.
Classify the person that answered these questions into one of the following categories representing where they are in their grief journey:

{categories}

Respond with a number representing the appropriate category. Respond with only the number and no other text.

Answers to survey questions:
{qanda}

Category number:
"""

In [316]:
categories = ""
for stage in journey_map:
  categories += f"{journey_map[stage]['number']}. {stage}:" + journey_map[stage]['description'] + "\n"
categories = categories.strip()
print(categories)

1. Immediate Grief, Shock & Emotion:Overwhelmed, loss of purpose; shock and trauma emotions (isolation) present and challenging to understand. Individuals may struggle to deal with family responsibilities alone. Surviving Child: Feeling disconnected without guidance and attention from grieving adults.
2. Navigating Family Relationships:Experiencing tension between individuals within the family unit; lack of support from family members. Surviving Family Unit: Perception of other family members’ grief experience. Each family member may be at different phases of their grief journey.
3. Learning to Process Grief:Experiencing grief and learning to process those emotions. Surviving Child: Seeks guidance and acknowledgment of grief; benefit from opportunities to open up and process with kids in similar situations to normalize emotions.
4. Moments That Matter:Renewed experience of grief around anniversaries of loss, holidays, and special moments. Surviving Family Unit: Navigating special momen

In [317]:
messages = [
    {
        "role": "user",
        "content": prompt.format(
            categories=categories,
            qanda=qanda
        )
    }
]

result = client.chat.completions.create(
    model="Hermes-2-Pro-Llama-3-8B",
    messages=messages,
    max_tokens=1000
)
result['choices'][0]['message']['content']

'5'

# Chaining + Augmentation: Summarize then categorize

In [318]:
survey_num = 0

qanda = ""
for i in range(9, 21):
  qanda += questions[f'q{i}'].split('agreement: ')[-1] + ": " + rating_reference[data.loc[survey_num][f'q{i}']] + "\n"
for i in range(21, 23):
  qanda += questions[f'q{i}'] + ": " + str(data.loc[survey_num][f'q{i}'])
qanda = qanda.strip()

In [319]:
print(qanda)

I have a positive outlook toward life: Somewhat likely
I have short and/or long range goals: Extremely likely / Very likely
I feel all alone: Somewhat unlikely
I can see possibilities in the midst of difficulties: Neither likely nor unlikely / Neutral
I have faith that gives me comfort: Extremely likely / Very likely
I feel scared about my future: Somewhat unlikely
I can recall happy/joyful times: Somewhat likely
I have deep inner strength: Somewhat likely
I am able to give and receive care/love: Extremely likely / Very likely
I have a sense of direction: Extremely likely / Very likely
I believe that each day has potential: Extremely likely / Very likely
I feel my life has value and worth: Extremely likely / Very likely
Please share with TAPS your favorite moment of the weekend? Did you have a breakthrough moment this weekend you would like to share?: nanPlease share any additional feedback you have regarding your TAPS Seminar experience.: nan


In [320]:
prompt = """Below are a set of answers to questions from a survey of people going through a grief journey after losing a family member.
Summarize the current state of grief of the person that answered these questions in a few sentences.

Answers to survey questions:
{qanda}

Summarize of the state of grief:
"""

In [321]:
messages = [
    {
        "role": "user",
        "content": prompt.format(
            qanda=qanda
        )
    }
]

result = client.chat.completions.create(
    model="Hermes-2-Pro-Llama-3-8B",
    messages=messages,
    max_tokens=1000
)
summary = result['choices'][0]['message']['content']
summary

'The person answering these questions seems to be in a moderately stable and hopeful state of grief. They have positive outlooks on life and have strong goals both short and long-term. The respondent seems to have a sense of inner strength and faith that provides comfort, along with a belief in the value and worth of their life. Despite some feelings of being alone, the individual feels capable of giving and receiving love, and they believe that each day has potential. The person appears to be moving forward through their grief journey and finding some hope in the midst of their challenges.'

In [322]:
prompt = """Below is a summary of the state of a person going through a grief journey after losing a family member.
Classify the person that answered these questions into one of the following categories representing where they are in their grief journey:

{categories}

Respond with a number representing the appropriate category. Respond with only the number and no other text.

Summary of the particular person's state:
{summary}

Category number:
"""

In [323]:
messages = [
    {
        "role": "user",
        "content": prompt.format(
            categories=categories,
            summary=summary
        )
    }
]

result = client.chat.completions.create(
    model="Hermes-2-Pro-Llama-3-8B",
    messages=messages,
    max_tokens=1000
)
result['choices'][0]['message']['content']

'4'